In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import glob
import nd2reader
import numpy as np
from cytoolz import compose
import skimage
from skimage.color import rgb2gray
from skimage.feature import blob_dog, blob_log, blob_doh
from paulssonlab.projects.molecule_counting.segmentation import segment, invert
from paulssonlab.projects.molecule_counting.matriarch_stub import permute_labels

# import cv2
from matplotlib.offsetbox import TextArea, DrawingArea, OffsetImage, AnnotationBbox
import signal
import time

mpl.rcParams["figure.figsize"] = (10, 10)

In [ ]:
segment_phase = compose(segment, invert)

In [ ]:
# Set up timers so that functions timeout and move on if something is weird and they don't run properly.
# Should diagnose this issue properly eventually, but for testing this should be ok.
class TimeoutException(Exception):  # Custom exception class
    pass


def timeout_handler(signum, frame):  # Custom signal handler
    raise TimeoutException


# Change the behavior of SIGALRM
signal.signal(signal.SIGALRM, timeout_handler)

In [ ]:
filenames = glob.glob("/n/scratch2/jqs1/200228/*.nd2") + glob.glob(
    "/n/scratch2/jqs1/200305/*.nd2"
)
for i in range(len(filenames)):
    print(i, filenames[i])

In [ ]:
file = filenames[2]
print("file", file)
nd2 = nd2reader.ND2Reader(file)
phase_img = nd2.get_frame_2D(v=0, c=0, z=0)[1000:1600, 500:1200]
fluor_img = nd2.get_frame_2D(v=0, c=1, z=0)[1000:1600, 500:1200]
seg = segment_phase(phase_img)
fig, axes = plt.subplots(1, 3, sharex=True, sharey=True)
ax = axes.ravel()
ax[0].imshow(phase_img)
ax[1].imshow(fluor_img, cmap="RdGy")
ax[2].imshow(seg)

In [ ]:
def get_indices_in_r(ref_indices, r):
    # ref_indices is a list of the indices that define the location of the reference point.
    # r is the radius within which we want to collect indices.
    ranges = np.arange(np.floor(-r), np.ceil(r) + 1)
    rel_points = np.where(
        (ranges[np.newaxis, :]) ** 2 + (ranges[:, np.newaxis]) ** 2 < r**2
    )
    points = [
        rel_points[0] + ref_indices[0] - np.ceil(r),
        rel_points[1] + ref_indices[1] - np.ceil(r),
    ]
    return points

In [ ]:
def rdf(
    phase_img, fluor_img, cell_indices=[0], time_allowed=80, filename="20200603rdf"
):
    #     nd2 = nd2reader.ND2Reader(img_file)
    #     print(img_file)
    #     for fov_index in fov_indices:
    #         phase_img = nd2.get_frame_2D(v=0, c=0, z=fov_index)
    #         fluor_img = nd2.get_frame_2D(v=0, c=1, z=fov_index)
    #         print('phase and fluor done')
    # Segment the image.
    seg = segment_phase(phase_img)
    print("seg done")
    min_fluor = np.amin(fluor_img)
    print("minfluor", min_fluor)
    # Get number of cells in the image.
    cells = np.amax(seg)
    print("number of cells", cells)
    for cell_index in cell_indices:
        if cell_index > cells:
            print("Error in rdf: cell_index larger than number of cells in fov.")
            break
        cell = np.ma.masked_where(seg != cell_index + 1, fluor_img)
        print("max fluor in cell", cell[222, 172])
        print("mean", np.mean(cell))
        # Normalise the intensities of the cell.
        cell = (cell - min_fluor) / np.mean(cell)
        print("new cell", cell[222, 172])
        ref_point = np.where(cell == np.amax(cell))
        print("ref_point", ref_point)
        # rho is the average 'particle' density, where a particle corresponds to an intensity unit.
        print("sum cell", np.sum(cell))
        print("non zero elements", np.count_nonzero(cell.mask == 0))
        # rho = np.sum(cell)/np.count_nonzero(cell.mask==0)
        rho = np.mean(cell)
        print("rho", rho)
        r = 2
        mask = np.copy(cell.mask)
        gr_values = []
        while not np.all(mask):
            indices_in_r = get_indices_in_r(ref_point, r)
            #             print('indicesinr',indices_in_r)
            # Calculate numerator (gr_num) and volume element (gr_vol)
            #             print(cell[list(map(int, indices_in_r[0])), list(map(int, indices_in_r[1]))])
            gr_num = np.sum(
                cell[list(map(int, indices_in_r[0])), list(map(int, indices_in_r[1]))]
            )
            print(gr_num)
            gr_vol = len(indices_in_r[0])
            print(gr_vol)
            cell.mask[
                list(map(int, indices_in_r[0])), list(map(int, indices_in_r[1]))
            ] = True
            print(np.count_nonzero(cell.mask == 0))
            gr_values.append(gr_num / gr_vol)
            r += 10
        # Since rho is constant for all r, divide all at the same time to save on number of computations.
        gr_values = np.array(gr_values)
        print(gr_values)
        gr_values = np.divide(gr_values, rho)
        print(gr_values)
        break


rdf(phase_img, fluor_img)

In [ ]:
def get_stdevs_alpha_time_cellnorm(
    img_file, fov_indices, time_allowed=80, savetxt=False, filename=None
):
    # Normalises each image. Subtract background (non-cell) and
    # then divide by the mean fluorescence in the specific cell.
    nd2 = nd2reader.ND2Reader(img_file)
    stdevs_alpha = []
    fluor_diffs = []
    for fov_index in fov_indices:
        signal.alarm(time_allowed)
        try:
            print(str(img_file), "fov index", fov_index)
            phase_img = nd2.get_frame_2D(v=0, c=0, z=fov_index)
            fluor_img = nd2.get_frame_2D(v=0, c=1, z=fov_index)
            # Segment the image.
            seg = segment_phase(phase_img)
            min_fluor = np.amin(fluor_img)
            # Get number of cells in the image.
            cells = np.amax(seg)
            # Get the standard deviation in fluorescence of each cell in the image.
            for i in range(cells):
                cell = np.ma.masked_where(seg != i + 1, fluor_img)
                cell = (cell - min_fluor) / np.mean(cell)
                stdevs_alpha.append(np.std(cell))
                fluor_diffs.append(np.amax(cell) - np.amin(cell))
            #                 print(np.amin(cell),np.amax(cell), np.amax(cell)-np.amin(cell),np.std(cell))
            if savetxt:
                np.savetxt(filename + "_alpha_cellnorm.csv", stdevs_alpha)
                np.savetxt(filename + "_fluordiffs_cellnorm.csv", fluor_diffs)
        except TimeoutException:
            if savetxt:
                np.savetxt(filename + "_alpha_cellnorm.csv", stdevs_alpha)
                np.savetxt(filename + "_fluordiffs_cellnorm.csv", fluor_diffs)
            print("timeout")
            continue  # continue the for loop if function A takes more than 5 second
        else:
            # Reset the alarm
            signal.alarm(0)
    return stdevs_alpha

In [ ]:
fig, axes = plt.subplots(4, 5, figsize=(30, 15), sharex=True)
ax = axes.ravel()
logmeans = []
logstds = []
for i in range(len(files_names)):
    try:
        data_stdevs = np.loadtxt(
            "202004_cellnorm/" + files_names[i] + "_alpha_cellnorm.csv"
        )
        data_fluordiffs = np.loadtxt(
            "202004_cellnorm/" + files_names[i] + "_fluordiffs_cellnorm.csv"
        )
        ax[i].hist(data_stdevs, bins=np.logspace(-1.5, 0.5, 50))
        #         ax[i].hist(data_fluordiffs, bins = 50)
        ax[i].set_title(files_names[i])
        plt.xscale("log")
        print(files_names[i], "number of cells:", len(data_stdevs))
        print(files_names[i], "mean of stdevs:", np.mean(data_stdevs))
        print(files_names[i], "stdev of stdevs:", np.std(data_stdevs))
        print(
            files_names[i],
            "mean of log10 stdevs:",
            np.mean(np.log10(data_stdevs)),
            10 ** (np.mean(np.log10(data_stdevs))),
        )
        logmeans.append(10 ** (np.mean(np.log10(data_stdevs))))
        print(
            files_names[i],
            "stdev of log10 stdevs:",
            np.std(np.log10(data_stdevs)),
            10 ** (np.std(np.log10(data_stdevs))),
        )
        logstds.append(10 ** (np.std(np.log10(data_stdevs))))
    except OSError:
        pass

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 10))
ax = axes.ravel()
for i in range(len(logstds)):
    if logstds[i] < 1.7:
        ax[0].plot(logmeans[i], logstds[i], "bo")
    else:
        ax[0].plot(logmeans[i], logstds[i], "ro")
ax[0].axhline(1.7, linestyle="--", color="k")
for i in range(len(logstds)):
    if logstds[i] < 1.7:
        ax[1].plot(logmeans[i], logstds[i], "bo")
    else:
        ax[1].plot(logmeans[i], logstds[i], "ro")
ax[1].axhline(1.7, linestyle="--", color="k")
for i, txt in enumerate(files_names):
    ax[1].annotate(txt, (logmeans[i] + 0.001, logstds[i] + 0.05))
plt.xlabel("Mean of cell stdevs")
plt.ylabel("Stdev of cell stdevs")

In [ ]:
for i in range(len(files_names)):
    if logstds[i] < 1.7:
        print(files_names[i])

In [ ]:
files_names = [
    "chr_P_sfGFP",
    "chr_P_GFPmut3",
    "chr_P_Venus",
    "chr_P_Dendra2",
    "plas_X_DHL_GFPmut3",
    "plas_P_DHL_sfGFP40to54",
    "plas_X_arai_sfGFP",
    "chr_P_GFPmut3_40x",
    "chr_P_sfGFP_40x",
    "chr_P_Venus_40x",
    "chr_P_Dendra2_40x",
    "chr_P_sfGFP_20x",
    "chr_P_GFPmut3_20x",
    "chr_P_Dendra2_20x",
    "chr_P_Venus_20x",
    "plas_X_DHL_GFPmut3_20x",
    "plas_X_DHL_GFPmut3_40x",
    "plas_P_DHL_sfGFP_20x",
    "plas_P_DHL_sfGFP_40x",
]

fig, axes = plt.subplots(3, 1, figsize=(3, 9), sharex=True)
ax = axes.ravel()
# names = ['chr_P_sfGFP', 'chr_P_sfGFP_40x', 'chr_P_sfGFP_20x']
# names = ['chr_P_GFPmut3', 'chr_P_GFPmut3_40x', 'chr_P_GFPmut3_20x']
# names = ['chr_P_Venus', 'chr_P_Venus_40x', 'chr_P_Venus_20x']
# names = ['chr_P_Dendra2', 'chr_P_Dendra2_40x', 'chr_P_Dendra2_20x']
# names = ['plas_X_DHL_GFPmut3', 'plas_X_DHL_GFPmut3_40x', 'plas_X_DHL_GFPmut3_20x']
# names = ['plas_P_DHL_sfGFP40to54', 'plas_P_DHL_sfGFP_40x', 'plas_P_DHL_sfGFP_20x']

base = names[1][:-4]
ax[0].set_title(base + " 100x")
ax[1].set_title(base + " 40x")
ax[2].set_title(base + " 20x")

for i in range(len(names)):
    try:
        data_stdevs = np.loadtxt("202004_cellnorm/" + names[i] + "_alpha_cellnorm.csv")
        data_fluordiffs = np.loadtxt(
            "202004_cellnorm/" + names[i] + "_fluordiffs_cellnorm.csv"
        )
        ax[i].hist(data_stdevs, bins=np.logspace(-1.5, 0.5, 50))
        plt.xscale("log")
    except OSError:
        pass
fig.savefig(base)